In [43]:
%matplotlib inline

# Hartree–Fock in 1D Using FEM with Lagrange Polynomials on Gauss–Lobatto Nodes

Each finite element is defined in the local coordinate system $\xi \in [-1, 1]$.
Using Gauss–Lobatto–Legendre (GLL) quadrature, integrals are approximated over $N$ nodes as:

$$
\int_{-1}^{1} f(\xi)\, d\xi \;\approx\; \sum_{i=0}^{N-1} w_i\, f(\xi_i),
$$

where $\xi_i, w_i$ are the GLL nodes and weights corresponding to a polynomial of degree $N - 1$.
The routines below generate these pairs.
## Routines to generate GLL nodes and weights

### 1. Fast NumPy version

In [44]:
import numpy as np
from numpy.polynomial.legendre import Legendre

def GLL_nodes_weights_numpy(N):
    """Gauss–Lobatto–Legendre nodes x and weights w on (-1,1) obtained with numpy.
    N : int   – number of nodes (>=2)
    """
    P = Legendre.basis(N - 1)
    xi = P.deriv().roots()
    x  = np.hstack((-1.0, xi, 1.0))
    w0 = 2 / (N * (N - 1))
    w  = np.hstack((w0, 2 / (N * (N - 1) * P(xi)**2), w0))
    return x, w

### 2. Custom version based on Newton–Raphson method with user‑defined tolerance

In [45]:
def GLL_nodes_weights_custom(N, tol=1e-6, max_iter=50):
    """
    Gauss–Lobatto–Legendre nodes and weights on (-1, 1) obtained with specified tolerance.

    N : int   – number of nodes (>=2)
    tol : float – convergence tolerance for Newton iterations
    max_iter : int – safety limit for Newton iterations
    """
    if N < 2:
        raise ValueError("N must be at least 2")
    if N == 2:
        return np.array([-1., 1.]), np.ones(2)

    # Initial guess with Chebyshev–Lobatto nodes
    nodes = np.cos(np.pi * np.arange(N - 1, -1, -1) / (N - 1))

    # column is the degree of poly and row for inner nodes. P_j(x_i), i=1..N-1
    P = np.empty((N - 2, N))
    P[:, 0] = 1.0
    for _ in range(max_iter):
        x_old = nodes[1:-1].copy()

        P[:, 0] = 1.0
        P[:, 1] = x_old
        for k in range(2, N):
            P[:, k] = ((2*k - 1) * x_old * P[:, k - 1] -
                       (k - 1) * P[:, k - 2]) / k

        P_N   = P[:, -1]
        P_Nm1 = P[:, -2]
        nodes[1:-1] -= (nodes[1:-1] * P_N - P_Nm1) / ((N + 1) * P_N)

        if np.max(np.abs(nodes[1:-1] - x_old)) < tol:
            break
    else:
        raise RuntimeError(f"GLL iteration did not converge in {max_iter} steps")

    if (np.diff(np.sort(nodes)) < 1e-12).any():
        raise ValueError("Duplicated roots detected in GLL procedure")

    weights = np.empty(N)
    weights[1 : -1] = 2.0 / ((N - 1) * N * P_N**2)
    weights[0] = weights[-1] = 2 / (N * (N - 1))

    return nodes, weights

## FEM_1Dgrid class

FEM_1Dgrid stores all geometry and quadrature related data needed for one‑dimensional FEM calculations with Lagrange interpolating polynomials (LIPs) defined on GLL nodes. In addition, the class has solvers for single‑particle and two‑particle problems with user-defined external and interparticle potentials.

### 1. Global‑matrix assembly options

The class supports two strategies for assembling element matrices into global matrices, controlled by the `grid_type` parameter,
which can be either `'naive'` or `'real'` according to [HelFEM](https://doi.org/10.1002/qua.25945) (Eq.13).
The difference between the two assembly schemes is illustrated in the figure below:

![Two implemented ways of global matrices assembling](data/global_matrices_order.jpg)

### 2. Coordinate Mapping

In addition to the standard linear mesh, the class also provides a logarithmic mesh suitable for Coulomb potentials.
This behavior is controlled by the `x_scale_type` parameter, which can be set to `'linear'` or `'log'`.

**Linear mesh**
Each element has constant Jacobian $J_e$:

$$
x(\xi) = \frac{x_i + x_{i+1}}{2} + \frac{x_{i+1} - x_i}{2} \, \xi,
\qquad
J_e = \frac{dx}{d\xi} = \frac{x_{i+1} - x_i}{2}.
$$

**Logarithmic mesh** – uniform in $u=\ln x$

$$
u(\xi)=\frac{u_i+u_{i+1}}{2}+\frac{u_{i+1}-u_i}{2}\,\xi,
\qquad
x(\xi)=e^{u(\xi)},
\qquad
J_e(\xi) = \frac{dx}{d\xi}=\frac{u_{i+1}-u_i}{2}\,x(\xi).
$$

In [46]:
class FEM_1Dgrid:
    def __init__(self, N_elements, degree, xmin, xmax,
                 grid_type='real', two_e_method_type='full', x_scale_type='linear'):
        """
        Initializes the grid and parameters for solving the equation FEM.

        Parameters:
        tol : float - Tolerance for both orthogonality and residual tests.
        N_elements : int   - Number of finite elements.
        degree     : int   - Degree of LIPs.
        xmin, xmax : float - Left and right boundaries.

        grid_type  : str   - Grid type for matrix construction.
            'naive' – elementwise matrices are used without assembling (Eq. 13),
            'real'  – matrices are assembled globally with node identification (Eq. 14).

        two_e_method_type : str - Specifying method for calculation of two-electron integrals
            'full'  - implementation copied from https://github.com/lnw/1d-experiments
            'diag'  - g_diag[i, k] = g4[i,i,j,j]

        x_scale_type      : str - Scaling of the x-coordinate ('linear' or 'log').
        """
        self.N_elements        = N_elements
        self.degree            = degree
        self.N_nodes           = degree + 1
        self.grid_type         = grid_type.lower()
        self.two_e_method_type = two_e_method_type.lower()
        self.x_scale_type      = x_scale_type.lower()

        if self.grid_type == 'real':
            self.N_points = N_elements * degree + 1
        elif self.grid_type == 'naive':
            self.N_points = N_elements * self.N_nodes
        else:
            raise ValueError("type_grid must be 'real' or 'naive'")

        if self.x_scale_type == 'linear':
            self.elem_borders = np.linspace(xmin, xmax, N_elements + 1)
        elif self.x_scale_type == 'log':
            if xmin <= 0:
                raise ValueError("xmin must be positive for scale='log'")
            ln_edges = np.linspace(np.log(xmin), np.log(xmax), N_elements + 1)
            self.elem_borders = np.exp(ln_edges)
        else:
            raise ValueError("scale must be 'linear' or 'log'")

        self.xi_nodes, self.w_nodes = GLL_nodes_weights_custom(self.N_nodes, tol=1e-15)

        self.build_elements_connectivity()
        self.L  = self.local_lagr_vals()
        self.dL = self.local_lagr_derivs()

### 3. Quadrature matrices prepared by the constructor
`x_quad[e,q]` - Global cordinate of node $q$ in element $e$ with shape (Ne, N),

`wJ[e,q]` - GLL weights multiplied by Jacobian with shape (Ne, N),

`w_over_J[e,q]` - GLL weight devided by Jacobian with shape (Ne, N),

`x_global[i]` - global coordinates containing GLL nodes with shape (Np),

`w_global[i]` - vector of weights for each global coordinate with shape (Np).

*Ne - number of elements, Np - total number of nodes.

In [47]:
def build_elements_connectivity(self):
    """ Build all element‑dependent geometric arrays.
    x_quad[e, q] – x coordinate in element e,
    wJ[e, q]     - quadrature weight (w_q) times J[e],
    w_over_J[e, q] = w_q / J_{e,q},
    x_global     – 1‑D array with global x coordinates
                   (with duplication when grid_type='naive');
    w_global[i]    – global integration weights  𝑤_q * J[e,q]
    """
    self.x_global = np.zeros(self.N_points)
    self.w_global = np.zeros(self.N_points)
    self.x_quad   = np.zeros((self.N_elements, self.N_nodes))
    self.wJ       = np.zeros((self.N_elements, self.N_nodes))
    self.w_over_J = np.zeros((self.N_elements, self.N_nodes))

    for e in range(self.N_elements):
        xL, xR = self.elem_borders[e], self.elem_borders[e + 1]

        if self.x_scale_type == "linear":
            # affine map  x = x0 + J_e * xi with constant Jacobian
            J_e = 0.5 * (xR - xL)
            self.x_quad[e] = 0.5 * (xR + xL) + J_e * self.xi_nodes
            self.wJ[e] = self.w_nodes * J_e
            self.w_over_J[e] = self.w_nodes / J_e

        elif self.x_scale_type == "log":
            # linear map in u = ln x, then x = exp(u)
            uL, uR = np.log(xL), np.log(xR)
            Ju = 0.5 * (uR - uL)
            u0 = 0.5 * (uR + uL)

            u_nodes = u0 + Ju * self.xi_nodes
            self.x_quad[e] = np.exp(u_nodes)
            J_nodes = Ju * self.x_quad[e]  # dx/dxi at each node
            self.wJ[e] = self.w_nodes * J_nodes
            self.w_over_J[e] = self.w_nodes / J_nodes
        # -------- fill global coordinate array -------------------------
        if self.grid_type == "real":
            idx = slice(e * self.degree, e * self.degree + self.N_nodes)
            self.x_global[idx] = self.x_quad[e]
            self.w_global[idx] += self.wJ[e]
        else:  # 'naive'
            idx = slice(e * self.N_nodes, (e + 1) * self.N_nodes)
            self.x_global[idx] = self.x_quad[e]
            self.w_global[idx] += self.wJ[e]
            
FEM_1Dgrid.build_elements_connectivity = build_elements_connectivity

### 4. Values and derivatives of the nodal Lagrange polynomials

For Gauss‑Lobatto nodes $\xi_q$ the $N$ LIPs $\chi_i(\xi)$ are defined by the nodal property

$$
\chi_i(\xi_q)=\delta_{iq}\ \qquad (i,q=0,\dots,N-1).
$$

Hence the matrix of values $L_{qi}\equiv\chi_i(\xi_q)$ is the identity matrix $I_{N}$.
Derivatives $d\chi_i/d\xi$ do not form an identity and must be computed explicitly.


In [48]:
def local_lagr_vals(self):
    L = np.ones((self.N_nodes, self.N_nodes))
    for i in range(self.N_nodes):
        for j in range(self.N_nodes):
            if i != j:
                L[:, i] *= (self.xi_nodes - self.xi_nodes[j]) / \
                           (self.xi_nodes[i] - self.xi_nodes[j])
    return L

def local_lagr_derivs(self):
    dL = np.zeros((self.N_nodes, self.N_nodes))
    for i in range(self.N_nodes):
        for k in range(self.N_nodes):
            if k == i:
                continue
            term = 1.0 / (self.xi_nodes[i] - self.xi_nodes[k])
            for j in range(self.N_nodes):
                if j != i and j != k:
                    term *= (self.xi_nodes - self.xi_nodes[j]) / \
                            (self.xi_nodes[i] - self.xi_nodes[j])
            dL[:, i] += term
    return dL

FEM_1Dgrid.local_lagr_vals = local_lagr_vals
FEM_1Dgrid.local_lagr_derivs = local_lagr_derivs

### Calculation of local matrices and assembling global matrices

For every element $(e)$ we compute three local matrices $S^{(e)}, T^{(e)}, V^{\text{ext},(e)}$ with shape $N \times N$ each. Because the nodal Lagrange functions satisfy $\chi_i(\xi_q) \chi_j(\xi_q)=\delta_{iq} \delta_{jq}$, the matrices take the following form

$$
\begin{aligned}
S_{ij}^{(e)} &=
\int_{-1}^{1}\chi_i(\xi)\,\chi_j(\xi)\,J^{(e)}(\xi)\,d\xi
\;\approx\;\sum_{q=0}^{N-1} w_q\,J^{(e)}(\xi_q)\,
\chi_i(\xi_q)\,\chi_j(\xi_q)
= w_i\,J^{(e)}_i\,\delta_{ij}
\\[6pt]
T_{ij}^{(e)} &=
\frac{1}{2}\int_{-1}^{1}
\frac{d\chi_i}{dx}\frac{d\chi_j}{dx}\,J^{(e)}(\xi)\,d\xi
= \frac{1}{2}\int_{-1}^{1}
\frac{d\chi_i}{d\xi}\frac{d\chi_j}{d\xi}\frac{1}{J^{(e)}(\xi)}\,d\xi
\;\approx\;\frac{1}{2}\sum_{q=0}^{N-1}
\frac{w_q}{J^{(e)}_q}\chi_i'(\xi_q)\,\chi_j'(\xi_q)
\\[6pt]
V^{\text{ext},(e)}_{ij} &=
\int_{-1}^{1}\chi_i(\xi)\,\chi_j(\xi)\,V^{\text{ext}}(x(\xi))\,
J^{(e)}(\xi)\,d\xi
\;\approx\;\sum_{q=0}^{N-1}
w_q\,J^{(e)}(\xi_q)\,V^{\text{ext}}(x_q)\,
\chi_i(\xi_q)\,\chi_j(\xi_q)
= w_i\,J^{(e)}_i\,V^{\text{ext}}(x_i)\,\delta_{ij}
\end{aligned}
$$

The local blocks $S^{(e)},T^{(e)},V^{(e)}$ are then inserted into the global matrices according to the chosen assembly strategy ('naive' or 'real').


In [49]:
def assemble_global(self, V_ext=None):
    " Assemble_global  (S, T, V). "
    T = np.zeros((self.N_points, self.N_points))
    S = np.zeros_like(T)
    V = np.zeros_like(T) if V_ext is not None else None

    for e in range(self.N_elements):
        # local matrices with element‑specific weights
        S_loc = self.L.T @ np.diag(self.wJ[e]) @ self.L
        T_loc = 0.5 * self.dL.T @ np.diag(self.w_over_J[e]) @ self.dL

        # global slice
        if self.grid_type == 'real':
            if e < self.N_elements - 1:
                idx = slice(e * self.degree, e * self.degree + self.N_nodes)
            else:
                idx = slice(e * self.degree, e * self.degree + self.N_nodes)
        else:
            idx = slice(e * self.N_nodes,  (e + 1) * self.N_nodes)

        S[idx, idx] += S_loc
        T[idx, idx] += T_loc

        if V_ext is not None:
            Vdiag = V_ext(self.x_quad[e]) * self.wJ[e]
            V_loc = self.L.T @ np.diag(Vdiag) @ self.L
            V[idx, idx] += V_loc

    self.S, self.T = S, T
    if V_ext is not None:
        self.V = V

FEM_1Dgrid.assemble_global = assemble_global

### Two‑electron integrals

With a user‑defined interaction $V^{ee}(|x-x'|)$ the four‑index integral is

$$
g_{ijkl} = \iint \chi_i(x)\,\chi_j(x) V^{ee}(|x-x'|) \chi_k(x')\,\chi_l(x')\,dx\,dx' \approx\ \sum_{q,q'} w_q w_{q'} V^{ee}(|x_q-x_{q'}|) \delta_{iq}\,\delta_{jq}\,\delta_{kq'}\,\delta_{lq'} = w_i w_k\,V^{ee}(|x_i-x_k|)\,\delta_{ij}\,\delta_{kl}.
$$

Thus only the diagonal block $g_{ik} \equiv g_{i i k k}$ is needed.

In [50]:
def build_two_e_diag(self, V_ee_func):
    """
    Compute the diagonal two-electron block g[ii,kk]:
        g_diag[i, k] = w_i w_k J_e(i) J_e(k) V_ee(|x_i - x_k|)
    """
    X = np.abs(self.x_global[:, None] - self.x_global[None, :])
    self.g_diag = np.outer(self.w_global, self.w_global) * V_ee_func(X)

FEM_1Dgrid.build_two_e_diag = build_two_e_diag

Another implementation of 2e integral is `build_two_e_full` copied from [https://github.com/lnw/1d-experiments](https://github.com/lnw/1d-experiments)) produces the same numerical result as `build_two_e_diag`, but `build_two_e_full` has shape $Np \times Np \times Np \times Np$. The diagonal shortcut is faster and lighter on memory.

In [51]:
def build_two_e_full(self, V_ee_func):
    """
    Replica of calc_and_set_4c2e_lip(...) from https://github.com/lnw/1d-experiments.
    Computes self.g4 with shape (Np,Np,Np,Np).
    The integral is

        (mu nu | lambda sigma)
          = sum_{k in cell_munu} w_k J_munu  L_kmu L_knu
            * sum_{r in cell_lasi} w_r J_lasi
                V(|x_k - x_r|) L_rlambda L_rsigma
    """
    munulambdasigma = np.zeros((self.N_elements, self.N_elements,
                                self.N_nodes, self.N_nodes, self.N_nodes, self.N_nodes))

    #  Loop over the first element cell_munu
    for cell_munu in range(self.N_elements):

        # buffer: int_lambda_sigma[k, cell_lasi, lam, sig]
        buf = np.zeros((self.N_nodes, self.N_elements, self.N_nodes, self.N_nodes))

        # inner loop: x1
        for k in range(self.N_nodes):
            # Global coordinate of node k in element cell_munu
            x1 = self.x_quad[cell_munu, k]

            # scan all second elements
            for cell_lasi in range(self.N_elements ):
                for lam in range(self.N_nodes ):
                    for sig in range(self.N_nodes ):
                        acc = 0.0
                        # integrate over x2 (nodes r in cell_lasi)
                        for r in range(self.N_nodes ):
                            x2 = self.x_quad[cell_lasi, r]
                            r12 = abs(x1 - x2)

                            acc += (self.wJ[cell_lasi, r] *
                                    V_ee_func(r12) *
                                    self.L[r, lam] * self.L[r, sig])
                        buf[k, cell_lasi, lam, sig] = acc

        # outer contraction over x1
        for cell_lasi in range(self.N_elements):
            for mu in range(self.N_nodes):
                for nu in range(self.N_nodes):
                    for lam in range(self.N_nodes):
                        for sig in range(self.N_nodes):
                            acc = 0.0
                            for k in range(self.N_nodes):
                                acc += (self.wJ[cell_munu, k] *
                                        buf[k, cell_lasi, lam, sig] *
                                        self.L[k, mu] * self.L[k, nu])
                            munulambdasigma[cell_munu, cell_lasi, mu, nu, lam, sig] = acc

    #  Convert to global tensor shape (Npoints,Npoints,Npoints,Npoints)
    self.g4 = np.zeros((self.N_points, self.N_points,
                        self.N_points, self.N_points))

    def elem_slice(e):
        """Return global‑node slice for element e."""
        if self.grid_type == 'real':
            return slice(e * self.degree, e * self.degree + self.N_nodes)
        elif self.grid_type == 'naive':
            return slice(e * self.N_nodes, (e + 1) * self.N_nodes)

    for cell_munu in range(self.N_elements):
        sl_mu = elem_slice(cell_munu)
        for cell_lasi in range(self.N_elements):
            sl_la = elem_slice(cell_lasi)
            block = munulambdasigma[cell_munu, cell_lasi]

            # accumulate – required if boundary nodes overlap
            self.g4[sl_mu, sl_mu, sl_la, sl_la] += block

FEM_1Dgrid.build_two_e_full = build_two_e_full

Next method `apply_boundaries` enforces the boundary conditions by deleting the first and last rows and columns.

In [52]:
def apply_boundaries(self):
    "Removes first and last columns and rows."
    keep = slice(1, self.N_points - 1)
    self.T = self.T[keep, keep]
    self.S = self.S[keep, keep]
    if hasattr(self, 'V'):
        self.V = self.V[keep, keep]
    if hasattr(self, 'g_diag'):
        self.g_diag = self.g_diag[keep, keep]
    if hasattr(self, 'g4'):
        self.g4 = self.g4[keep, keep, :, :]
        self.g4 = self.g4[:, :, keep, keep]

FEM_1Dgrid.apply_boundaries = apply_boundaries

### Function for diagonalization of the single‑particle Hamiltonian
Eigenvectors are normalised on demand.

In [53]:
import scipy.linalg as la

def solve(self, V_ext, norm_flag=True):
    self.assemble_global(V_ext)
    self.apply_boundaries()

    H = self.T + (self.V if hasattr(self, 'V') else 0)
    E, C = la.eigh(H, self.S)
    self.E = E
    if norm_flag:
        SC = self.S @ C
        norms = np.sqrt(np.sum(C * SC, axis=0))

        # avoid division by zero
        norms[norms == 0.0] = 1.0
        C /= norms
    self.C = C

FEM_1Dgrid.solve = solve

### RHF self‑consistent field procedure (`scf_rhf`)
Steps of calculation:
1. **Core integrals**
   $H_{\text{core}} = T + V,\qquad S_{ij}=\langle\chi_i|\chi_j\rangle$.

2. **Two‑electron integrals** according to `two_e_method_type` flag.
3. **Orthogonalisation**
   Solve $USU^{\mathrm T}=s$; set $X = U\,\text{diag}(s^{-1/2})\,U^{\mathrm T}$.

4. **SCF loop**

   * Density: $P_{ij}=2\sum_{p\in\text{occ}} C_{ip}C_{jp}$.
   * Coulomb/Exchange
     $J_{ij}= \sum_{kl} g_{ij,kl}P_{kl},\;
       K_{ij}= \sum_{kl} g_{ik,lj}P_{kl}$
     (diagonal shortcuts when requested).
   * Fock: $F = H_{\text{core}} + J - \tfrac12 K$.
   * Energy $E_{\text{tot}} = \operatorname{Tr}[PH_{\text{core}}] + \tfrac12\operatorname{Tr}[PJ] - \tfrac14\operatorname{Tr}[PK].$

   Iterate until $|\Delta E_{\text{tot}}|$ and norm of $\Delta P$ drop below `conv`.

5. **Stored results**
   eigenvalues `self.E`, orbitals `self.C`, density `self.P`, total energy `self.E_tot`.

In [54]:
def scf_rhf(self, V_ext_func, V_ee_func, occ_idx_list,
            max_iter=100, conv=1e-8, verbose=True):
    """
    RHF

    V_ext_func : callable - External one‑electron potential V(x).
    V_ee_func : callable -  Two‑electron interaction V_ee(|x1‑x2|).
    occ_idx_list : list[int] - Indices of doubly occupied orbitals, origin from 0.
    max_iter : int - Maximum SCF iterations.
    conv : float - Convergence threshold for dE and dP_norm.
    verbose : bool - If True, print iteration log.
    """
    # build core matrices
    self.assemble_global(V_ext_func)
    if self.two_e_method_type == 'diag':
        self.build_two_e_diag(V_ee_func)
    elif self.two_e_method_type == 'full':
        self.build_two_e_full(V_ee_func)

    self.apply_boundaries()

    H_core = self.T + self.V

    # for orthogonalization
    s, U = la.eigh(self.S)
    X = U @ np.diag(1 / np.sqrt(s)) @ U.T

    # initial guess
    F = H_core.copy()
    eps, C2 = la.eigh(X.T @ F @ X)
    C = X @ C2

    # initial density
    P = sum(2 * np.outer(C[:, i], C[:, i]) for i in occ_idx_list)
    E_prev = np.inf

    for it in range(1, max_iter + 1):
        # build J and K
        if self.two_e_method_type == 'diag':
            rho_vec = np.diag(P)
            J = np.diag(self.g_diag @ rho_vec)
            K = self.g_diag * P
        else:
            J = np.einsum('ijkl,kl->ij', self.g4, P)
            K = np.einsum('ikjl,kl->ij', self.g4, P)

        F = H_core + J - 0.5 * K
        eps, C2 = la.eigh(X.T @ F @ X)
        C = X @ C2
        P_new = sum(2 * np.outer(C[:, i], C[:, i]) for i in occ_idx_list)

        # energies
        E_one = np.sum(P_new * H_core)
        E_coul = 0.5 * np.sum(P_new * J)
        E_exch = -0.25 * np.sum(P_new * K)
        E_tot = E_one + E_coul + E_exch

        dE = E_tot - E_prev
        dP = np.linalg.norm(P_new - P) / np.sqrt(P.size)

        if verbose:
            print(f"iter {it:3d}:  E = {E_tot:+.12f}  dE = {dE:+.2e}  dP_norm = {dP:.2e}")

        if abs(dE) < conv and dP < conv:
            if verbose:
                print("SCF converged.")
            break

        P = P_new
        E_prev = E_tot
    else:
        raise RuntimeError("SCF did not converge")

    # store results
    self.E = eps
    self.C = C
    self.E_tot = E_tot
    
FEM_1Dgrid.scf_rhf = scf_rhf

### Function to make plots

In [55]:
from matplotlib import cm
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider

def plot_wavefunction_interactive(self, n=1, log_scale=False,
                                  V_func=None, analyt_func=None, n_origin=0):
    fig, (ax_wf, ax_rho) = plt.subplots(2, 1, sharex=True,
                                        figsize=(7, 5), constrained_layout=True)
    X = np.log(self.x_global) if log_scale else self.x_global
#     colour = cm.get_cmap('tab10')(0)
    colour = plt.colormaps['tab10'](0)

    # numeric (alpha)
    psi = np.zeros(self.N_points)
    psi[1:-1] = self.C[:, n - n_origin]
    ax_wf.plot(X, psi, "-", color=colour, lw=1.5,
               label=f"numeric  n={n},  E={self.E[n - n_origin]:.4f}")
    ax_rho.plot(X, psi**2, "-", color=colour, alpha=0.8)

    # analytic
    if analyt_func is not None:
        ana = analyt_func(self.x_global, n)
        ax_wf.plot(X, ana, ":", color='crimson', lw=1.2, label="analytic")
        ax_rho.plot(X, ana**2, ":", color='crimson', alpha=0.7)

    # optional potential
    if V_func is not None:
        for ax in (ax_wf, ax_rho):
            axV = ax.twinx()
            axV.plot(X, V_func(self.x_global), "--", lw=1.0, color="grey")
            axV.set_ylabel("V(x)", color="grey")
            axV.tick_params(axis='y', labelcolor="grey")

    ax_wf.set_ylabel(r"Psi(x)")
    ax_rho.set_ylabel("Density")
    ax_rho.set_xlabel("log x" if log_scale else "x")
    ax_wf.legend()
    plt.show()
    
FEM_1Dgrid.plot_wavefunction_interactive = plot_wavefunction_interactive

# One particle tests
## One particle in a harmonic potential
The problem can be solved accurately only with `grid_type`=`real`; for `grid_type`=`naive` the solver is failing.

In [56]:
from scipy.special import hermite, factorial

V_harm = lambda x: 0.5 * x**2

grid = FEM_1Dgrid(N_elements=200, degree=3, xmin=-10, xmax=10, grid_type='real')
grid.solve(V_harm)
for n in range(10):
    print(f"n={n} E_num={grid.E[n]: 6.4e} E_harm={0.5 + n : 6.4e} delta={grid.E[n] - (0.5 + n): 6.4e}")

n=0 E_num= 5.0000e-01 E_harm= 5.0000e-01 delta=-1.1154e-11
n=1 E_num= 1.5000e+00 E_harm= 1.5000e+00 delta=-9.8141e-11
n=2 E_num= 2.5000e+00 E_harm= 2.5000e+00 delta=-4.4516e-10
n=3 E_num= 3.5000e+00 E_harm= 3.5000e+00 delta=-1.4000e-09
n=4 E_num= 4.5000e+00 E_harm= 4.5000e+00 delta=-3.4832e-09
n=5 E_num= 5.5000e+00 E_harm= 5.5000e+00 delta=-7.3897e-09
n=6 E_num= 6.5000e+00 E_harm= 6.5000e+00 delta=-1.3987e-08
n=7 E_num= 7.5000e+00 E_harm= 7.5000e+00 delta=-2.4317e-08
n=8 E_num= 8.5000e+00 E_harm= 8.5000e+00 delta=-3.9595e-08
n=9 E_num= 9.5000e+00 E_harm= 9.5000e+00 delta=-6.1210e-08


In [57]:
Psi_harmonic = lambda x, n: 1.0 / np.sqrt((2**n) * factorial(n) * np.sqrt(np.pi)) * hermite(n)(x) * np.exp(-0.5 * x**2)
interact(lambda n: grid.plot_wavefunction_interactive(
        n=n,
        V_func=V_harm,
        analyt_func=Psi_harmonic,
        log_scale=False,
        ),
        n=IntSlider(min=0, max=10, step=1, value=1, description='n'))


interactive(children=(IntSlider(value=1, description='n', max=10), Output()), _dom_classes=('widget-interact',…

<function __main__.<lambda>(n)>

## One particle in a Coulomb potential  

We check the logarithmic grid by solving the 1‑D Coulomb problem.
The results are not as accurate as expected; this could mean there is a bug in the code or that we need finer grid settings.

In [60]:
Z = 1.0
V_coul = lambda x: -Z / np.abs(x)
E_coul = lambda n: -0.5 * (Z / n)**2


grid = FEM_1Dgrid(N_elements=100, degree=2, xmin=1e-05, xmax=200,
                  grid_type='real', x_scale_type='log')
grid.solve(V_coul, norm_flag=True)

for n in range(1, 10):
    print(f"n={n} E_num={grid.E[n - 1]: 6.4e} E_coul={E_coul(n): 6.4e} delta={grid.E[n - 1] - E_coul(n): 6.4e}")

n=1 E_num=-4.9998e-01 E_coul=-5.0000e-01 delta= 2.1833e-05
n=2 E_num=-1.2499e-01 E_coul=-1.2500e-01 delta= 6.6915e-06
n=3 E_num=-5.5547e-02 E_coul=-5.5556e-02 delta= 8.1806e-06
n=4 E_num=-3.1240e-02 E_coul=-3.1250e-02 delta= 1.0273e-05
n=5 E_num=-1.9985e-02 E_coul=-2.0000e-02 delta= 1.4785e-05
n=6 E_num=-1.1328e-02 E_coul=-1.3889e-02 delta= 2.5612e-03
n=7 E_num=-9.5345e-03 E_coul=-1.0204e-02 delta= 6.6956e-04
n=8 E_num=-8.5758e-03 E_coul=-7.8125e-03 delta=-7.6325e-04
n=9 E_num=-6.7291e-03 E_coul=-6.1728e-03 delta=-5.5626e-04


In [61]:
from scipy.special import genlaguerre

def R_nl(r, n, l=0, Z=1.0):
    rho = 2 * Z * r / n
    coeff = np.sqrt((2 * Z / n)**3 * factorial(n - l - 1) / (2 * n * factorial(n + l)))
    L = genlaguerre(n - l - 1, 2 * l + 1)
    return coeff * rho**l * np.exp(-rho / 2) * L(rho) * r

interact(lambda n: grid.plot_wavefunction_interactive(
        n=n,
        V_func=V_coul,
        analyt_func=R_nl,
        log_scale=True,
        n_origin=1),
        n=IntSlider(min=1, max=10, step=1, value=1, description='n'))

interactive(children=(IntSlider(value=1, description='n', max=10, min=1), Output()), _dom_classes=('widget-int…

<function __main__.<lambda>(n)>

## Two particles problems
To reproduce results obtained with [https://github.com/lnw/1d-experiments](https://github.com/lnw/1d-experiments), the following potentials are used

$$
\begin{aligned}
V^{\text{ext}} &= \frac{1}{2} x^2 - 100;    V^{\text{ee}}  &= \exp(-0.15 x^2)
\end{aligned}
$$


In [62]:
V_ext = lambda x: 0.5 * x**2 - 100.0
V_ee  = lambda x, a=0.15: np.exp(-a * x**2)

grid = FEM_1Dgrid(N_elements=100, degree=2, xmin=-10, xmax=10,
                  grid_type='real', x_scale_type='linear', two_e_method_type='diag')

occ_idces = [0]          # doubly‑occupied orbital indices (starting from 0)
grid.scf_rhf(V_ext_func=V_ext,
             V_ee_func = V_ee,
             occ_idx_list = occ_idces,
             max_iter = 100,
             verbose = True)
print("HF energy =", grid.E_tot)

iter   1:  E = -198.123348305279  dE = -inf  dP_norm = 4.08e-03
iter   2:  E = -198.127297444427  dE = -3.95e-03  dP_norm = 1.03e-03
iter   3:  E = -198.128175032082  dE = -8.78e-04  dP_norm = 2.63e-04
iter   4:  E = -198.128389638061  dE = -2.15e-04  dP_norm = 6.72e-05
iter   5:  E = -198.128443806994  dE = -5.42e-05  dP_norm = 1.72e-05
iter   6:  E = -198.128457615974  dE = -1.38e-05  dP_norm = 4.41e-06
iter   7:  E = -198.128461147774  dE = -3.53e-06  dP_norm = 1.13e-06
iter   8:  E = -198.128462052164  dE = -9.04e-07  dP_norm = 2.89e-07
iter   9:  E = -198.128462283868  dE = -2.32e-07  dP_norm = 7.41e-08
iter  10:  E = -198.128462343245  dE = -5.94e-08  dP_norm = 1.90e-08
iter  11:  E = -198.128462358462  dE = -1.52e-08  dP_norm = 4.87e-09
iter  12:  E = -198.128462362362  dE = -3.90e-09  dP_norm = 1.25e-09
SCF converged.
HF energy = -198.12846236236234


Output from  [https://github.com/lnw/1d-experiments](https://github.com/lnw/1d-experiments) with the same input:
```
./ho-1d 100 0.2 20 3 1.
```

The output:
```
system: n cells: 100
cell width: 0.2
lower boundary: -10
functions per cell: 3
overlapping: 1

grid: {{-0.1, 0, 0.1}, {0.033333..., 0.133333..., 0.033333...}}
grid fine: {{-0.1, -0.044721..., 0.044721..., 0.1}, {0.016666..., 0.083333..., 0.083333..., 0.016666...}}
...
HF energy:         -198.128458
xci energy:        -198.1351807
correlation energy:  -0.00672278
SCI (aka HF):      -198.128458
DCI:               -198.1351257
SDCI:              -198.1351807
````

**Comparison with the Python FEM implementation**

* HF energy (C++ reference) : **−198.128 458** Ha
* HF energy (our code)     : **−198.128 462 362 Ha**

The two values differ by only ≈ 4 × 10⁻⁶ Ha.
A small residual deviation is likely due to the fact that the C++ program two types of mesh—`grid` and `grid_fine`, whereas the current Python script uses only one grid, but I'm not sure.

In [63]:
interact(lambda n: grid.plot_wavefunction_interactive(
        n=n,
        V_func=V_ext,
        n_origin=1),
        n=IntSlider(min=1, max=10, step=1, value=1, description='n'))

interactive(children=(IntSlider(value=1, description='n', max=10, min=1), Output()), _dom_classes=('widget-int…

<function __main__.<lambda>(n)>

### Helium atom

With the Coulomb electron-electron interaction the SCF cycle does not converge:
the two-electron integral matrix haas infinite diagonal elements, and I currently do not know how to continue.

For now the calculation is carried out with the **soft-Coulomb** electron-electron potentail:

$$
V^{\text{ext}}(x)= -\frac{Z}{x}, \qquad
V^{\text{ee}}(x)=\frac{1}{\sqrt{x^{2}+a^{2}}}\; .
$$

Minimal value of $a$ at which the calculation converges $a = 0.5$.

In [65]:
V_ext = lambda x, Z=2.0: -Z / np.abs(x)
V_ee  = lambda r, a=0.5: 1.0 / np.sqrt(r**2 + a**2)

grid = FEM_1Dgrid(N_elements=100, degree=2, xmin=1e-5, xmax=300,
                  grid_type='real', x_scale_type='log', two_e_method_type='diag')

occ_idces = [0]
grid.scf_rhf(V_ext_func=V_ext, V_ee_func=V_ee, occ_idx_list=occ_idces, max_iter=100, verbose=True)
print("HF energy =", grid.E_tot)

iter   1:  E = -2.482774827159  dE = -inf  dP_norm = 2.24e-02
iter   2:  E = -2.518993616274  dE = -3.62e-02  dP_norm = 5.36e-03
iter   3:  E = -2.527877300797  dE = -8.88e-03  dP_norm = 1.16e-03
iter   4:  E = -2.529900811908  dE = -2.02e-03  dP_norm = 2.81e-04
iter   5:  E = -2.530381961665  dE = -4.81e-04  dP_norm = 7.61e-05
iter   6:  E = -2.530504975272  dE = -1.23e-04  dP_norm = 2.20e-05
iter   7:  E = -2.530538423555  dE = -3.34e-05  dP_norm = 6.59e-06
iter   8:  E = -2.530547925328  dE = -9.50e-06  dP_norm = 2.01e-06
iter   9:  E = -2.530550704956  dE = -2.78e-06  dP_norm = 6.18e-07
iter  10:  E = -2.530551534115  dE = -8.29e-07  dP_norm = 1.91e-07
iter  11:  E = -2.530551784709  dE = -2.51e-07  dP_norm = 5.92e-08
iter  12:  E = -2.530551861124  dE = -7.64e-08  dP_norm = 1.84e-08
iter  13:  E = -2.530551884570  dE = -2.34e-08  dP_norm = 5.71e-09
iter  14:  E = -2.530551891795  dE = -7.23e-09  dP_norm = 1.78e-09
SCF converged.
HF energy = -2.5305518917949783


At the **RHF/aug‑cc‑pVQZ** level the reference total energy of He is **−2.861 521 995 63 Ha**, whereas our 1‑D Python model with the soft‑Coulomb parameter $a = 0.5$ yields **−2.530 352 327 207 283 Ha**.

In [66]:
interact(lambda n: grid.plot_wavefunction_interactive(
        n=n,
        V_func=V_ext,
        n_origin=1,
        log_scale=True,
        ),
        n=IntSlider(min=1, max=10, step=1, value=1, description='n'))

interactive(children=(IntSlider(value=1, description='n', max=10, min=1), Output()), _dom_classes=('widget-int…

<function __main__.<lambda>(n)>